Computer vision

In [4]:
import cv2   #pip install opencv-contrib-python

In [5]:
video=cv2.VideoCapture('sample.mp4')

In [3]:
while True:
    state,frame=video.read()
    if not state:
        break
    cv2.imshow('video output',frame)
    if cv2.waitKey(1)==27:
        break
video.release()
cv2.destroyAllWindows()

In [8]:
def webcam(webcam_idx=0):
    video=cv2.VideoCapture(webcam_idx)
    while True:
        state,frame=video.read()
        if not state:
            break
        cv2.imshow('video output',frame)
        if cv2.waitKey(1)==27:
            break
video.release()
cv2.destroyAllWindows()

In [9]:
webcam()

In [1]:
def recorder(webcam_idx=0,recording_file='recording.avi'):
    video=cv2.VideoCapture(webcam_idx)
    fourcc=cv2.VideoWriter_fourcc(*'XVID')
    out=cv2.VideoWriter(recording_file,fourcc,20.0,(640,480))
    while True:
        state,frame=video.read()
        if not state:break
        cv2.imshow('video camera',frame)
        out.write(frame)
        if cv2.waitKey(1)==27:
            break
    out.release()
    video.release()
    cv2.destroyAllWindows()

In [6]:
recorder(recording_file='example_webcam.avi')

In [10]:
def webcam_w_info(webcam_idx=0):
    video=cv2.VideoCapture(webcam_idx)
    while True:
        state,frame=video.read()
        if not state:
            break
        cv2.rectangle(frame,(0,0),(500,20),(255,255,0),-1)
        cv2.putText(frame,"webcam output from laptop",(10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0))
        cv2.putText(frame,'camera o',(10,450),cv2.FONT_HERSHEY_SIMPLEX,.5,(0,255,255))
        cv2.imshow('video output',frame)
        if cv2.waitKey(1)==27:
            break
    video.release()
    cv2.destroyAllWindows()
webcam_w_info()


In [11]:
import mediapipe as np